# Imports

In [144]:
import pandas as pd
from sklearn.metrics import roc_curve

# Importando base

In [145]:
df_curr_full = pd.read_csv('application_data.csv')

In [146]:
df_prev_full = pd.read_csv('previous_application.csv')

# Join nas bases

In [147]:
df_joined = pd.merge(df_curr_full, df_prev_full, on='SK_ID_CURR', how='inner')

In [148]:
df_joined['IDENT_KEY'] = df_joined.index

In [149]:
df_joined.head(5)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE_x,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY_x,...,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,IDENT_KEY
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,24.0,low_normal,POS other with interest,365243.0,-565.0,125.0,-25.0,-17.0,0.0,0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,12.0,low_normal,Cash X-Sell: low,365243.0,-716.0,-386.0,-536.0,-527.0,1.0,1
2,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,6.0,middle,POS industry with interest,365243.0,-797.0,-647.0,-647.0,-639.0,0.0,2
3,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,12.0,middle,POS household with interest,365243.0,-2310.0,-1980.0,-1980.0,-1976.0,1.0,3
4,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,4.0,middle,POS mobile without interest,365243.0,-784.0,-694.0,-724.0,-714.0,0.0,4


In [150]:
list(df_joined.columns)

['SK_ID_CURR',
 'TARGET',
 'NAME_CONTRACT_TYPE_x',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT_x',
 'AMT_ANNUITY_x',
 'AMT_GOODS_PRICE_x',
 'NAME_TYPE_SUITE_x',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'OCCUPATION_TYPE',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'WEEKDAY_APPR_PROCESS_START_x',
 'HOUR_APPR_PROCESS_START_x',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'ORGANIZATION_TYPE',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGIN

## Criando valor da parcela e entrada

In [151]:
df_joined['INSTALLMENT'] = (0.7 * df_joined['AMT_CREDIT_x'])/(df_joined['CNT_PAYMENT'])

In [152]:
df_joined['ENTRY'] = 0.3*df_joined['AMT_CREDIT_x']

# Renomeando colunas

In [153]:
dic = {
    'IDENT_KEY': 'CPF',
    'TARGET' : 'BAD',
    'AMT_CREDIT_x': 'DEBT_VALUE',
    'CNT_PAYMENT' : 'INSTALLMENT_CNT'
}

In [154]:
df_joined = df_joined.rename(columns=dic)

In [155]:
df_joined.head(1)

,SK_ID_CURR,BAD,NAME_CONTRACT_TYPE_x,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,DEBT_VALUE,AMT_ANNUITY_x,...,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,CPF,INSTALLMENT,ENTRY
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,POS other with interest,365243.0,-565.0,125.0,-25.0,-17.0,0.0,0,11859.09375,121979.25


# Selecionando variáveis necessárias para conversação com Bot

In [156]:
vars_nec = ['CPF','DEBT_VALUE','INSTALLMENT_CNT', 'INSTALLMENT', 'ENTRY']

# Seleção de variáveis

## Teste de Ks
    - No contexto de um problema com um alvo binário, o teste de KS ajuda a avaliar o poder discriminatório de diferentes variáveis preditoras, analisando a diferença máxima entre as funções de distribuição acumulada de cada variável para as duas categorias do alvo.
    - Vamos focar nas variáveis que têm maior poder discriminatório, isto é, aquelas que apresentam um valor KS elevado. Com elas vai ser possível constuir nosso modelo final que vai ajudar a dizer a probabilidade de determinado valor de acordo ser honrado, dado o contexto do devedor. 

In [157]:
vars_drop = vars_nec
all_columns = set(df_joined.columns)
columns_excluding_vars_drop = list(all_columns.difference(vars_drop + ['BAD']))
BAD_col = "BAD"

ks_stats = []

for col in columns_excluding_vars_drop:
    if pd.api.types.is_numeric_dtype(df_joined[col]):
        fpr, tpr, thresholds = roc_curve(df_joined[BAD_col], df_joined[col].fillna(df_joined[col].mean()))
        ks_statistic = max(tpr - fpr)
        ks_stats.append((col, ks_statistic))
    else:
        ks_stats.append((col, None)) 

ks_stats = sorted(ks_stats, key=lambda x: x[1] if x[1] is not None else -1, reverse=True)

In [158]:
model_cols = [col_name for col_name, corr_val in ks_stats if corr_val is not None and abs(corr_val) > 0.03]
for col_name in vars_nec:
    model_cols.append(col_name)
len(model_cols)

40

In [159]:
model_cols

['DAYS_BIRTH',
 'DAYS_LAST_PHONE_CHANGE',
 'DAYS_LAST_DUE_1ST_VERSION',
 'DAYS_ID_PUBLISH',
 'DAYS_LAST_DUE',
 'DAYS_TERMINATION',
 'DAYS_FIRST_DUE',
 'DAYS_EMPLOYED',
 'REG_CITY_NOT_WORK_CITY',
 'FLAG_EMP_PHONE',
 'DAYS_DECISION',
 'FLAG_DOCUMENT_3',
 'DAYS_REGISTRATION',
 'REGION_RATING_CLIENT_W_CITY',
 'REGION_RATING_CLIENT',
 'NFLAG_INSURED_ON_APPROVAL',
 'LIVE_CITY_NOT_WORK_CITY',
 'NONLIVINGAREA_AVG',
 'NONLIVINGAREA_MEDI',
 'NONLIVINGAREA_MODE',
 'OWN_CAR_AGE',
 'AMT_ANNUITY_x',
 'FLAG_WORK_PHONE',
 'REG_CITY_NOT_LIVE_CITY',
 'DEF_30_CNT_SOCIAL_CIRCLE',
 'CNT_CHILDREN',
 'NONLIVINGAPARTMENTS_MODE',
 'AMT_REQ_CREDIT_BUREAU_DAY',
 'NONLIVINGAPARTMENTS_MEDI',
 'NONLIVINGAPARTMENTS_AVG',
 'AMT_REQ_CREDIT_BUREAU_YEAR',
 'AMT_REQ_CREDIT_BUREAU_HOUR',
 'AMT_REQ_CREDIT_BUREAU_WEEK',
 'ENTRANCES_MODE',
 'AMT_GOODS_PRICE_y',
 'CPF',
 'DEBT_VALUE',
 'INSTALLMENT_CNT',
 'INSTALLMENT',
 'ENTRY']

# Construindo base que vai ser usada no treinamento 

In [160]:
df_final = df_joined[model_cols]